In [2]:
import numpy as np
import pandas as pd
import spotipy as sp
import numbers
import spotipy.util as util

SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'
SPOTIPY_CLIENT_SECRET='c02f97ddd119470d8e24b290722c7490'
SPOTIPY_CLIENT_ID='647ffb8c163b4882be0cafe4bf1352d5'
USERNAME='lh7smhcywr18vctg4xmnoehzu'
SCOPE='user-library-read'

token = util.prompt_for_user_token(username=USERNAME,
                           scope=SCOPE,
                           client_id=SPOTIPY_CLIENT_ID,
                           client_secret=SPOTIPY_CLIENT_SECRET,
                           redirect_uri=SPOTIPY_REDIRECT_URI)

s = sp.Spotify(auth=token)

In [3]:
def get_track_ids(tracks, ide=True):
    """Collect all track id's in a list of tracks"""
    
    if ide:
        return [track['track']['id'] for track in tracks]
    else:
        return [track['id'] for track in tracks]


def get_audio_features(ids):
    """Collect all audio features in a list from a given list of track id's"""
    audio_feats = []

    for i in range(0, len(ids), 100):
        audio_feats += s.audio_features(ids[i:i + 100])

    return audio_feats

def getDataframe(features):
    matrix = []
    metamatrix = []
    for track in features:
        try:
            matrix += [[x for x in track.values() if isinstance(x, numbers.Number)]]
            metamatrix += [[x for x in track.values() if not isinstance(x, numbers.Number)]]
        except:
            pass
        
    return np.array(matrix), metamatrix



def get_random_sample(genre, iters = 20):
    """Main function"""
    query = s.search(q='genre:{0}'.format(genre))
    tracks = []
    
    i = 0
    
    while query['tracks']['next']:
        tracks.extend(query['tracks']['items'])
        query = s.next(query['tracks'])
        
        i += 1
        
        if i == iters:
            break
            
    feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
  
    # These steps are not computationally expensive
    track_ids = get_track_ids(tracks, ide=False)
    audio_feats = get_audio_features(track_ids)
    
    matrix_of_features, metamatrix = getDataframe(audio_feats)
    df = pd.DataFrame(matrix_of_features)
    df.columns = feature_names
    
    return df, metamatrix

In [4]:
# Laurens: vaporwave, classical,    hiphop, house
# Pieter:  rock,      punk,         jazz,   funk
# Douwe: metal,       meditation,   k-pop, hardcore
# Nima: rap,          soul,        techno, r-n-b
a1, b1 = get_random_sample("rap", 1000)
a2, b2 = get_random_sample("soul", 1000)
a3, b3 = get_random_sample("techno", 1000)
a4, b4 = get_random_sample("grunge", 1000)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [5]:
a1.to_pickle('rap_features')
a2.to_pickle('soul_features')
a3.to_pickle('techno_features')
a4.to_pickle('grunge_features')

In [6]:
import pickle

pickle.dump(b1, open('rap_metafeatures', 'wb'))
pickle.dump(b2, open('soul_metafeatures', 'wb'))
pickle.dump(b3, open('techno_metafeatures', 'wb'))
pickle.dump(b4, open('grunge_metafeatures', 'wb'))